# JGR model Gillespie simulation in numba

In [1]:
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline
from scipy.integrate import trapz
import numba
import time 

    
@numba.jit(nopython=True)
def prop(n,m,params,gam,kap):
    """
    Returns an array of propensities given a set of parameters
    and an array of populations.
    n -- population of moving grains
    m -- population of stationary grains
    params -- (nu,lam,sig,mu)
    gam -- migration out 
    kap -- coupling between bed elevation and transport
    """
    nu,lam,sig,mu = params
    dm = kap*m
    mp = 1+dm
    if mp>2:
        mp=2
    mm = 1-dm
    if mm<0:
        mm=0
    #out = np.array([nu,(lam+mu*n)*(1+dm), sig*n*(1-dm), gam*n])
    out = np.array([nu,(lam+mu*n)*mp, sig*n*mm, gam*n])    
    return out

@numba.jit(nopython=True)
def sum_numba(ar):
    """sum numbers fast"""
    return ar.sum()

@numba.jit(nopython=True)
def sample(probs, probs_sum):
    """draw a random element from the discrete distribution probs/probs_sum
    probs is an N-dimensional array and probs_sum = sum(probs)"""
    q = np.random.rand() * probs_sum 
    i = 0
    p_sum = 0.0
    while p_sum < q:
        p_sum += probs[i]
        i += 1
    return i - 1

@numba.jit(nopython=True)
def sim(kap,params, tmax=1000): 
    """
    kap -- coupling between transport and bed elevation on 0<kap<1
    params -- nu,lam,sig,mu (migration in, entrainment, deposition, collective entrainment)
    tmax -- max time of the simulation in 1/[units of the rates] (i.e. seconds)
    This function returns n,m,t 
    n -- sequence of # moving grains
    m -- sequence of # stationary grains (relative to the mean)
    t -- sequence of transition times
    """
    nu, lam, sig, mu = params # all but gamma are specified
    gam = nu/lam*(sig-mu) # computed by conservation of mass at z=0
    #O = []
    n = int((nu+lam)/(sig+gam-mu))
    m = 0   
    t = 0.0
    N=[]
    M=[]
    T=[]
    while t<tmax:
        #T.append(t) # build up the output
        #O.append((n,m,t))
        N.append(n)
        M.append(m)
        T.append(t)
        # select the transition
        a = prop(n,m,params,gam,kap) # compute the propensities
        a0 = sum_numba(a) # find the net propensity 
        t = t + (1/a0)*np.log(1/np.random.random()) # step the time
        j=sample(a,a0) # index of the transition
        # enact the transition
        if j==1: # sorted by what I assume are most likely
            n = n+1
            m = m-1
        elif j==2:
            n = n-1
            m = m+1
        elif j==0:
            n = n+1
        else: # j==3
            n = n-1
    # shape the output
    #n = np.array([s[0] for s in O])
    #m = np.array([s[1] for s in O])
    #t = np.array([s[2] for s in O])
    #return n,m,t
    return np.array(N),np.array(M),np.array(T)

# perform all paper simulations

In [ ]:
params_a = (5.45, 6.59, 4.67,3.74) # flow a
params_g = (7.74,8.42,4.95,4.34)
params_i = (15.56,22.07,4.52,3.56) # flow i 
params_l = (15.52,14.64,4.77,4.32)
params_n = (15.45,24.49,3.64,4.24)
params = [params_a, params_g, params_i, params_l, params_n]
a = 0.3
lvals = np.linspace(1,10,13)*a
L = 22.5
phi = 0.6
z1 = np.pi*a**2/phi/L
kaps = z1**2/(2*lvals)**2
tmax = 100*3600
i=-1
j = len(kaps)*len(params)
for p in params:
    for kap in kaps:
        i+=1
        if i>51:
            print(str(i) + ' of ' + str(j))
            n,m,t = sim(kap,p,tmax=tmax)
            np.save('sim-'+str(i),{'parameters':p,'kappa':kap,'n':n,'m':m,'t':t})
            del n,m,t 

In [ ]:
@numba.jit(nopython=True)
def pdfer(q,dt,tm):
    qvals = np.unique(q)
    return np.array([qvals,[dt[q==qq].sum()/tm for qq in qvals]])
@numba.jit(nopython=True)
def binner(n,m,mvals,dt,o):
    return np.array([(n[m==mm]**o*dt[m==mm]).sum()/dt[m==mm].sum() for mm in mvals])
@numba.jit(nopython=True)
def momenter(q,o,dt,tm):
    return (q**o*dt).sum()/tm

def analyze(i):
    name='sim-'+str(i)+'.npy'
    file=np.load(name,allow_pickle=True).item()
    name=name[:-4]+'-ana'
    n = file['n']
    m = file['m']
    t = file['t']
    p = file['parameters']
    kap = file['kappa']
    del file
    
    # caluclate mean of n and variance of n 
    dt = np.diff(t)
    tm = t[-1]
    n = n[:-1]
    #nbar = (n*dt).sum()/tm
    nbar = momenter(n,1,dt,tm)
    #nvar = (n**2*dt).sum()/tm-nbar**2
    nvar = momenter(n,2,dt,tm)-nbar**2
    del t 
    
    # calculate mean of m and variance of m 
    m = m[:-1]
    #mbar = (m*dt).sum()/tm
    mbar = momenter(m,1,dt,tm)
    #mvar = (m**2*dt).sum()/tm-mbar**2
    mvar = momenter(m,2,dt,tm)-mbar**2
    
    # save all of these to output
    out ={}
    out['parameters']=p
    out['kappa']=kap
    out['mbar']=mbar
    out['mvar']=mvar
    out['nbar']=nbar
    out['nvar']=nvar
    
    # now calculate the pdf of n and m
    #nvals = np.unique(n)
    #n_pdf = [dt[n==nn].sum()/tm for nn in nvals]
    #n_pdf = np.array([nvals,n_pdf])
    n_pdf = pdfer(n,dt,tm)
    out['n_pdf']=n_pdf
    del n_pdf
    #del n_pdf,nvals

    # and now the m pdf 
    #mvals = np.unique(m)
    #m_pdf = [dt[m==mm].sum()/tm for mm in mvals]
    #m_pdf = np.array([mvals,m_pdf])
    m_pdf = pdfer(m,dt,tm)
    out['m_pdf']=m_pdf
    mvals = m_pdf[0]
    del m_pdf

    # now get the mean and variance of n held marginal to m
    #n_bar_m = [(n[m==mm]*dt[m==mm]).sum()/dt[m==mm].sum() for mm in mvals]
    n_bar_m = binner(n,m,mvals,dt,1)
    out['n_bar_m']=np.array([mvals,n_bar_m])
    #n_var_m = [(n[m==mm]**2*dt[m==mm]).sum()/dt[m==mm].sum() for mm in mvals]
    n_var_m = binner(n,m,mvals,dt,2)-n_bar_m**2
    #n_var_m = np.array(n_var_m) - np.array(n_bar_m)**2
    out['n_var_m']=np.array([mvals,n_var_m])
    del n_bar_m,n_var_m
  
    np.save(name,out)
    

In [ ]:
for i in range(0,52):
    analyze(i)
    print(i)

In [ ]:
analyze(51)

# now compute all resting time pdfs 


In [2]:
def rtcdf(i):
    name='sim-'+str(i)+'.npy'
    name1='sim-'+str(i)+'-ana.npy'
    file=np.load(name,allow_pickle=True).item()
    file1=np.load(name1,allow_pickle=True).item()
    m,t = file['m'],file['t']
    mv,pm = file1['m_pdf']
    #del file, file1
    #bins = np.linspace(1e-3,1e7,100)
    bins = np.hstack((np.array([0]),np.geomspace(1e-7,1e7,num=150)))
    
    # generate masks of where entrainment and deposition events occur in the timeseries
    erodemask = m[1:]-np.roll(m,1)[1:]==-1  # indices into t[1:] where erosion occurred 
    depositmask = m[1:]-np.roll(m,1)[1:]==1 # indices into t[1:] where deposition occurred 
    te = t[1:][erodemask]
    td = t[1:][depositmask]
    me = m[1:][erodemask]
    md = m[1:][depositmask]
    inputs = (te,me,td,md,mv,pm)
    del mv,m,file,file1,name,name1,t,erodemask,depositmask
    
    def compute_crt(te, me, td, md, mv, bins = bins):
        """
        compute conditional return time given
        - entrainment timeseries te
        - bed elevations me at entrainment
        - deposition timeseries td
        - bed elevations md at deposition
        - a value mv at which to condition
        - a set of bins in time over which to histogram
        """
        returns = te[me==mv]
        departs = td[md==mv+1]

        if (len(returns)>1) and (len(departs)>1):
            while (returns[0]<departs[0]):
                returns=returns[1:]
            while (departs[-1]>returns[-1]):
                departs=departs[:-1]
            k = len(departs)-len(returns)
            if k>0:
                departs = departs[k:]
            elif k<0:
                returns = returns[:k]
        crt = returns - departs
        H,_ = np.histogram(crt, bins=bins)
        H = H.cumsum()
        if H.max()>0:
            H = 1.0-H/H.max()
        return H

    def compute_cdf(inputs, bins=bins):
        te, me, td, md, m, pm = inputs # these come from loader
        cdf = np.zeros(len(bins)-1,dtype=float) # sum into this iteratively
        for mv,p in zip(m, pm): # iterate through all mvalues in question
            c_cdf = compute_crt(te, me, td, md, mv)
            cdf+=c_cdf*p
        #data = np.array((bins[1:], cdf))
        return bins[1:],cdf
    bins, cdf = compute_cdf(inputs,bins=bins)
    return bins,cdf

In [3]:
import time
for i in range(39,52): 
    t0 = time.time()
    t,cdf = rtcdf(i)
    np.save('sim-'+str(i)+'-rt',np.array([t,cdf]))
    del t,cdf
    print(str(i) + ' in ' + str(round((time.time()-t0)/60,2)) + ' mins.')
    print('----------------------------------------------')

39 in 1.21 mins.
----------------------------------------------
40 in 1.55 mins.
----------------------------------------------
41 in 1.97 mins.
----------------------------------------------
42 in 2.16 mins.
----------------------------------------------
43 in 2.71 mins.
----------------------------------------------
44 in 2.92 mins.
----------------------------------------------
45 in 3.06 mins.
----------------------------------------------
46 in 3.32 mins.
----------------------------------------------
47 in 3.68 mins.
----------------------------------------------
48 in 4.29 mins.
----------------------------------------------
49 in 4.4 mins.
----------------------------------------------
50 in 4.82 mins.
----------------------------------------------
51 in 4.41 mins.
----------------------------------------------


In [ ]:
plt.loglog(t,cdf)

# investigate $\text{var}\{m\}$ vs $\kappa$

In [ ]:
params_i = (15.56,22.07,4.52,3.56) # flow i 
params_a = (5.45, 6.59, 4.67,3.74) # flow a
params_g = (7.74,8.42,4.95,4.34)
params_l = (15.52,14.64,4.77,4.32)


params=params_l

nu,lam,sig,mu = params
gam = nu/lam*(sig-mu) # this should make m0=mean(m).. this is important. 

kaps = np.geomspace(1e-6,5e-1,50)
nmeans = []
nvars = []
mmeans=[]
mvars = []
i=0
t0 = time.time()
for kap in kaps:
    n,m,t = sim(kap,params,tmax=24*3600)
    if np.any(np.abs(kap*m)>1):
        print('FAIL')
    nmeans.append(trapz(n,t)/t[-1])
    nvars.append(trapz(n**2,t)/t[-1]-(trapz(n,t)/t[-1])**2)
    mvars.append(trapz(m**2,t)/t[-1])
    mmeans.append(trapz(m,t)/t[-1])
    i+=1
    print('iteration {} in {} mins'.format(i,round((time.time()-t0)/60,2)))
    t0 = time.time()    

In [ ]:
kaps[42]

In [ ]:
plt.loglog(kaps,mvars,'x',markersize=10,label='SIM')
plt.loglog(kaps,1/kaps/4,color='black',label='MFT')
#plt.xlim(1e-6,1e-1)

In [ ]:
plt.loglog(kaps,nvars,'x')

In [ ]:
plt.loglog(kaps,nmeans,'x')

# keep the distinction straight: $\langle n \rangle = \sum_m \langle n | m\rangle$
# $\langle n | m \rangle $ decreases quasilinear with m
# $ \langle n^2 | m \rangle $ decreases quasilinear with m 
# $ \langle n \rangle $ is roughly constant with $\kappa$ provided $\kappa < 10^{-2}$
# $ \langle n^2 \rangle - \langle n \rangle^2$ is roughly constant under same conditions of small $\kappa$
# $\langle m \rangle = 0$ always
# $\langle m^2 \rangle = \frac{1}{4\kappa}$ except for large $\kappa \approx 1$
# you're left with two goals: get a MFT predicting $\sigma_m^2 = \frac{1}{4\kappa}$ and $\langle n^x | m \rangle \propto - m$
# the key approximations are (1) m sees static n and (2) $\kappa \ll 1 $



In [ ]:
plt.loglog(kaps,mvars,'x',markersize=10,label='SIM')
plt.loglog(kaps,1/kaps/4,color='black',label='MFT')
plt.title(r'MFT PREDICTS $\sigma_m^2 = \frac{1}{2\kappa}$')
plt.ylabel('VARIANCE IN M')
plt.xlabel('COUPLING')
plt.legend()
#plt.xlim(1e-6,1e-2)

# study $\text{var}\{n\}$ vs $\kappa$

In [ ]:
plt.loglog(kaps,nvars,'x')
#plt.ylim(0,max(nvars))
#plt.plot(kaps,1/kaps+min(nvars),color='black')

# investigate $\langle n | m \rangle $

In [ ]:
kap = 1e-3
params=params_i
n,m,t = sim(kap,params,tmax=24*3600)
dt = np.diff(t)/t[-1] # used to weight averages
n = n[:-1]
m = m[:-1]
del t
nu, lam, sig, mu = params # all but gamma are specified
gam = nu/lam*(sig-mu) # computed by conservation of mass at z=0
nbar = (nu+lam)/(sig+gam-mu)

In [ ]:
mvals = np.arange(-150,150)
nmeans = [(n[m==mi]*dt[m==mi]).sum()/dt[m==mi].sum() for mi in mvals]
#n2 = [(n[m==mi]**2*dt[m==mi]).sum()/dt[m==mi].sum() for mi in mvals]

In [ ]:
def condmean(mvals):
    dm = - kap*mvals
    return (lam*(1+dm)+nu)/(sig*(1-dm)+gam-mu*(1+dm))

In [ ]:
plt.plot(mvals,nmeans,'x')
plt.axvline(0,color='black')
plt.axhline(nbar,color='black')

#plt.plot(mvals,nbar - kap*mvals*nbar**2)
#plt.xlim(-3*m.std(),3*m.std())
#plt.ylim(-3*n.std(),3*n.std())
plt.plot(mvals,condmean(mvals))

In [ ]:
(np.abs(kap*m)>0).any()

ok conclusions on the mean suppression effect: It scales like kappa in some way I don't quite understand -- when kappa is relatively small the mean suppression is minimal, which means a proper mean field treatment should be totally good to go ... 

but at larger values of kappa there is a very strong variation of the mean with bed elevation. the mean flux is suppressed when the elevation is higher. 

# pure poisson only

In [ ]:
params= (1.0,15.0,3.4,0.0)

nu,lam,sig,mu = params
gam = nu/lam*(sig-mu) # this should make m0=mean(m).. this is important. 

kaps = np.geomspace(1e-6,5e-2,30)
nmeans = []
nvars = []
mmeans=[]
mvars = []
i=0
t0 = time.time()
for kap in kaps:
    n,m,t = sim(kap,params,tmax=5*24*3600)
    if np.any(np.abs(kap*m)>1):
        print('FAIL')
    nmeans.append(trapz(n,t)/t[-1])
    nvars.append(trapz(n**2,t)/t[-1]-(trapz(n,t)/t[-1])**2)
    mvars.append(trapz(m**2,t)/t[-1])
    mmeans.append(trapz(m,t)/t[-1])
    i+=1
    print('iteration {} in {} mins'.format(i,round((time.time()-t0)/60,2)))
    t0 = time.time()    

In [ ]:
plt.loglog(kaps,mvars,'x',markersize=10,label='SIM')
plt.loglog(kaps,1/kaps/2,color='black',label='MFT')
#plt.xlim(1e-6,1e-1)

In [ ]:
[((m[:-1]==mv)*np.diff(t)).

In [ ]:
params= (2.0,24.0,4.4,3.5)
nu,lam,sig,mu = params
gam = nu/lam*(sig-mu) # this should make m0=mean(m).. this is important.
kap = 1e-3
n,m,t = sim(kap,params,tmax=5*24*3600)


In [ ]:
(m[:-1]*dt).sum()/t[-1]

In [ ]:
mvals = np.arange(-150,150,5)
i = np.digitize(m,mvals)[:-1]-1
dt = np.diff(t)
means = [(n[:-1][ii==i]*dt[ii==i]).sum()/(dt[ii==i]).sum() for ii in np.unique(i)]
mvals = mvals[np.unique(i)]